In [1]:
import pandas as pd
import numpy as np
import joblib
from pystoned import CNLS, CNLSDDF
from pystoned.constant import CET_MULT, CET_ADDI, FUN_COST, FUN_PROD, RTS_VRS, RTS_CRS, RED_MOM, OPT_LOCAL

In [2]:
data = pd.read_csv("../data_ready/data_ready.csv").dropna()


In [3]:
data_hun = data[data["CNT"] == 'HUN']
data_pol = data[data["CNT"] == 'POL']
data_cze = data[data["CNT"] == 'CZE']
data_svk = data[data["CNT"] == 'SVK']

In [4]:
inputs = data[["total_students", "ratio_f2m", "ratio_t2s", "ratio_pt2ft"]].values.tolist()
outputs = data[["avg_math", "avg_cog", "avg_science", "avg_read"]]
d_outputs = data[["avg_math", "avg_cog", "avg_science", "avg_read"]].values.tolist()
undesirables = data[["dropout_rate"]].values.tolist()
contextuals = data[["bol_competition", "bol_location"]].values.tolist()
data["avg_scores"] = outputs.mean(axis = 1)
z_output = data[["avg_scores"]].values.tolist()

In [5]:
inputs_hun = data_hun[["total_students", "ratio_f2m", "ratio_t2s", "ratio_pt2ft"]].values.tolist()
outputs_hun = data_hun[["avg_math", "avg_cog", "avg_science", "avg_read"]]
d_outputs_hun = data_hun[["avg_math", "avg_cog", "avg_science", "avg_read"]].values.tolist()
undesirables_hun = data_hun[["dropout_rate"]].values.tolist()
contextuals_hun = data_hun[["bol_competition", "bol_location"]].values.tolist()
data_hun["avg_scores"] = outputs_hun.mean(axis = 1)
z_output_hun = data_hun[["avg_scores"]].values.tolist()


inputs_pol = data_pol[["total_students", "ratio_f2m", "ratio_t2s", "ratio_pt2ft"]].values.tolist()
outputs_pol = data_pol[["avg_math", "avg_cog", "avg_science", "avg_read"]]
d_outputs_pol = data_pol[["avg_math", "avg_cog", "avg_science", "avg_read"]].values.tolist()
undesirables_pol = data_pol[["dropout_rate"]].values.tolist()
contextuals_pol = data_pol[["bol_competition", "bol_location"]].values.tolist()
data_pol["avg_scores"] = outputs_pol.mean(axis = 1)
z_output_pol = data_pol[["avg_scores"]].values.tolist()

inputs_cze = data_cze[["total_students", "ratio_f2m", "ratio_t2s", "ratio_pt2ft"]].values.tolist()
outputs_cze = data_cze[["avg_math", "avg_cog", "avg_science", "avg_read"]]
d_outputs_cze = data_cze[["avg_math", "avg_cog", "avg_science", "avg_read"]].values.tolist()
undesirables_cze = data_cze[["dropout_rate"]].values.tolist()
contextuals_cze = data_cze[["bol_competition", "bol_location"]].values.tolist()
data_cze["avg_scores"] = outputs_cze.mean(axis = 1)
z_output_cze = data_cze[["avg_scores"]].values.tolist()

inputs_svk = data_svk[["total_students", "ratio_f2m", "ratio_t2s", "ratio_pt2ft"]].values.tolist()
outputs_svk = data_svk[["avg_math", "avg_cog", "avg_science", "avg_read"]]
d_outputs_svk = data_svk[["avg_math", "avg_cog", "avg_science", "avg_read"]].values.tolist()
undesirables_svk = data_svk[["dropout_rate"]].values.tolist()
contextuals_svk = data_svk[["bol_competition", "bol_location"]].values.tolist()
data_svk["avg_scores"] = outputs_svk.mean(axis = 1)
z_output_svk = data_svk[["avg_scores"]].values.tolist()

<ipython-input-5-9e91d9a0ad24>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_hun["avg_scores"] = outputs_hun.mean(axis = 1)
<ipython-input-5-9e91d9a0ad24>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_pol["avg_scores"] = outputs_pol.mean(axis = 1)
<ipython-input-5-9e91d9a0ad24>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [5]:
# define the CNLS-Z model
# model1 = CNLS.CNLS(
#     y = inputs, 
#     x = z_output, 
#     z = contextuals, 
#     cet = CET_ADDI, 
#     fun = FUN_PROD, 
#     rts = RTS_CRS
# )

# model2 = CNLS.CNLS(
#     y = inputs, 
#     x = z_output, 
#     z = contextuals, 
#     cet = CET_ADDI, 
#     fun = FUN_PROD, 
#     rts = RTS_VRS
# )

# define the CNLSDDF model with undesirable output
model3 = CNLSDDF.CNLSDDF(
    y = inputs, 
    x = d_outputs, 
    b = undesirables, 
    gb = -1.0,
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4 = CNLSDDF.CNLSDDF(
    y = inputs, 
    x = d_outputs, 
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)
# # display the coefficient of contextual variable

In [7]:
# define the CNLS-Z model
model1_hun = CNLS.CNLS(
    y = inputs_hun, 
    x = z_output_hun, 
    z = contextuals_hun, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_CRS
)

model2_hun = CNLS.CNLS(
    y = inputs_hun, 
    x = z_output_hun, 
    z = contextuals_hun, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_VRS
)

# define the CNLSDDF model with undesirable output
model3_hun = CNLSDDF.CNLSDDF(
    y = inputs_hun, 
    x = d_outputs_hun, 
    b = undesirables_hun, 
    gb = -1.0,
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4_hun = CNLSDDF.CNLSDDF(
    y = inputs_hun, 
    x = d_outputs_hun, 
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

In [8]:
# define the CNLS-Z model
model1_pol = CNLS.CNLS(
    y = inputs_pol, 
    x = z_output_pol, 
    z = contextuals_pol, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_CRS
)

model2_pol = CNLS.CNLS(
    y = inputs_pol, 
    x = z_output_pol, 
    z = contextuals_pol, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_VRS
)

# define the CNLSDDF model with undesirable output
model3_pol = CNLSDDF.CNLSDDF(
    y = inputs_pol, 
    x = d_outputs_pol, 
    b = undesirables_pol, 
    gb = -1.0,
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4_pol = CNLSDDF.CNLSDDF(
    y = inputs_pol, 
    x = d_outputs_pol, 
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)
# # display the coefficient of contextual variable

In [9]:
# define the CNLS-Z model
model1_cze = CNLS.CNLS(
    y = inputs_cze, 
    x = z_output_cze, 
    z = contextuals_cze, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_CRS
)

model2_cze = CNLS.CNLS(
    y = inputs_cze, 
    x = z_output_cze, 
    z = contextuals_cze, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_VRS
)

# define the CNLSDDF model with undesirable output
model3_cze = CNLSDDF.CNLSDDF(
    y = inputs_cze, 
    x = d_outputs_cze, 
    b = undesirables_cze, 
    gb = -1.0,
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4_cze = CNLSDDF.CNLSDDF(
    y = inputs_cze, 
    x = d_outputs_cze, 
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)
# # display the coefficient of contextual variable

In [10]:
# define the CNLS-Z model
model1_svk = CNLS.CNLS(
    y = inputs_svk, 
    x = z_output_svk, 
    z = contextuals_svk, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_CRS
)

model2_svk = CNLS.CNLS(
    y = inputs_svk, 
    x = z_output_svk, 
    z = contextuals_svk, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_VRS
)

# define the CNLSDDF model with undesirable output
model3_svk = CNLSDDF.CNLSDDF(
    y = inputs_svk, 
    x = d_outputs_svk, 
    b = undesirables_svk, 
    gb = -1.0,
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4_svk = CNLSDDF.CNLSDDF(
    y = inputs_svk, 
    x = d_outputs_svk, 
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

In [5]:
model1.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 470596          
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 2746            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.06    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 686
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.

In [7]:
model2.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 470596          
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 2746            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.11    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 686
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.

In [6]:
model3.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.


Error: rescode.err_invalid_aij(1473): The value of the A matrix element in constraint '' (215090) for variable 'x4806' (4805) is invalid.

In [14]:
# model4.optimize(OPT_LOCAL)

In [15]:
model1_hun.optimize(OPT_LOCAL)
model2_hun.optimize(OPT_LOCAL)
model3_hun.optimize(OPT_LOCAL)
model4_hun.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 12544           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 450             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 112
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.

In [26]:
model1_hun.ge

<bound method CNLS.get_frontier of <pystoned.CNLS.CNLS object at 0x7f353340a2b0>>

In [16]:
model1_pol.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 56169           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 950             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.01    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 237
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.

Error: rescode.err_invalid_aij(1473): The value of the A matrix element in constraint '' (1416) for variable 'x1663' (1662) is invalid.

In [ ]:
model2_pol.optimize(OPT_LOCAL)

In [ ]:
model3_pol.optimize(OPT_LOCAL)

In [ ]:
model4_pol.optimize(OPT_LOCAL)

In [17]:
### open ur eyes its not always poland. 
model1_cze.optimize(OPT_LOCAL)
model2_cze.optimize(OPT_LOCAL)
model3_cze.optimize(OPT_LOCAL)
model4_cze.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 38416           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 786             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.01    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 196
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.

In [18]:
model1_svk.optimize(OPT_LOCAL)
model2_svk.optimize(OPT_LOCAL)
model3_svk.optimize(OPT_LOCAL)
model4_svk.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 19881           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 566             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 141
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.